In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
import transformers
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml
from tqdm import tqdm
import logging
import json
import warnings
import random

logging.disable(logging.ERROR)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

model_name = 'fav-kky/FERNET-C5'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
BATCH_SIZE = 64

In [5]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
subj_path = '/home/horyctom/bias-detection-thesis/src/models/trained/subj_balanced.pth'
mb_path = '/home/horyctom/bias-detection-thesis/src/models/trained/mb_balanced.pth'
wikinpov_path = '/home/horyctom/bias-detection-thesis/src/models/trained/wiki_balanced.pth'
all_path = '/home/horyctom/bias-detection-thesis/src/models/trained/all_balanced.pth'

In [8]:
training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=50,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=0.1,
    learning_rate=5e-5)

In [9]:
token_full = preprocess_data(data,tokenizer,'sentence')

In [13]:
MODEL_PATH = subj_path

In [15]:
scores=[]
print("Running 5-fold CV on model: ",model_name,"...")
for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

    token_train = Dataset.from_dict(token_full[train_index])
    token_valid = Dataset.from_dict(token_full[val_index])

    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.load_state_dict(torch.load(MODEL_PATH))
    model.to(device)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
    trainer.train()

        #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
    print(scores[-1])
    
print(np.mean(scores))

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


Step,Training Loss
50,0.537000
100,0.333000


0.781352728468113


Step,Training Loss
50,0.548700
100,0.324300


0.8049844236760124


...

0.7938467891802603


Step,Training Loss
50,0.522200
100,0.336000


0.7611619150080688


Step,Training Loss
50,0.525400
100,0.329100


0.7554051823733288
0.7820338176979816


In [18]:
print("SUBJ: ",0.7820338176979816)

SUBJ:  0.7820338176979816


In [16]:
scores=[]
print("Running 5-fold CV on model: ",model_name,"...")
for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

    token_train = Dataset.from_dict(token_full[train_index])
    token_valid = Dataset.from_dict(token_full[val_index])

    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.load_state_dict(torch.load(wikinpov_path))
    model.to(device)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
    trainer.train()

        #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
    print(scores[-1])
    
print(np.mean(scores))

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


Step,Training Loss
50,0.485000
100,0.272200


0.7656152228547981


Step,Training Loss
50,0.503200
100,0.310000


0.8274601878317681


...

0.7747152995543819


Step,Training Loss
50,0.531400
100,0.345400


0.7583019160254093


Step,Training Loss
50,0.501300
100,0.293300


0.7651547178371467
0.7811561816293563


In [19]:
print("WIKI: ",0.7811561816293563)

WIKI:  0.7811561816293563


In [17]:
scores=[]
print("Running 5-fold CV on model: ",model_name,"...")
for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

    token_train = Dataset.from_dict(token_full[train_index])
    token_valid = Dataset.from_dict(token_full[val_index])

    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.load_state_dict(torch.load(mb_path))
    model.to(device)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
    trainer.train()

        #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
    print(scores[-1])
    
print(np.mean(scores))

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


Step,Training Loss
50,0.534800
100,0.290900


0.7780210292621257


Step,Training Loss
50,0.521900
100,0.296500


0.8178839381411729


...

0.7917831074035453


Step,Training Loss
50,0.521100
100,0.294900


0.7695356797303343


Step,Training Loss
50,0.533600
100,0.297100


0.7657344455530071
0.7747153006674318


In [20]:
print("WIKI: ",0.7747153006674318)

WIKI:  0.7747153006674318


In [10]:
scores=[]
print("Running 5-fold CV on model: ",model_name,"...")
for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

    token_train = Dataset.from_dict(token_full[train_index])
    token_valid = Dataset.from_dict(token_full[val_index])

    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.load_state_dict(torch.load(all_path))
    model.to(device)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
    trainer.train()

        #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
    print(scores[-1])
    
print(np.mean(scores))

Running 5-fold CV on model:  fav-kky/FERNET-C5 ...


Step,Training Loss
50,0.493300
100,0.273000


0.7782489501298862


Step,Training Loss
50,0.527500
100,0.320200


0.8270788999836307


...

0.7847589040249592


Step,Training Loss
50,0.496500
100,0.333100


0.7599700586351728


Step,Training Loss
50,0.491900
100,0.306400


0.7738193869096934
0.7804182795598127
